# Project: Investigate a Dataset - Medical Appointment No Shows

## Table of Contents
<ul>
<li><a href="#intro">1. Introduction</a></li>
<li><a href="#wrangling">2. Data Wrangling</a></li>
<li><a href="#eda">3. Exploratory Data Analysis</a></li>
<li><a href="#conclusions">4. Conclusions</a></li>
<li><a href="#references">5. References</a></li>
</ul>

<a id='intro'></a>
## 1. Introduction

This analysis is part of the Udacity Data Analysis Nanodegree Program and aims to explore the dataset of medical appointments no show in Brazil. This analysis is divided in four main parts:
1. *Introduction*, where the initial info is provided and the problem is set.
2. *Data Wrangling*, where the data is cleaned and prepared for analysis.
3. *Exploratory Data Analysis*, where key patterns are to be found.
4. *Conclusions*, in which the insights are described.


### Dataset Description 
This dataset contains over 100K medical appointments realized in Brazil. The information provided concerns The Brazil Public health system, known as SUS (*Sistema Único de Saúde*, Unified Health System), one of the largest health system in the world and entirely free of any cost. Being a system used by over 220 million Brazillians.

The dataset have 14 columns as so:
1. **PatientID**, Identification of a patient.
2. **AppointmentID**, Identification of each appointment.
3. **Gender**, "M" for Male and "F" for Female.
4. **ScheduledDay**, the day the patient set up the appointment.
5. **AppointmentDay**.
6. **Age**, age in years.
7. **Neighbourhood**, the location of the appointment.
8. **Scholarship**, whether the patient is enrolled in *Bolsa Família*.
9. **Hipertension**, True or False.
10. **Diabetes**, True or False.
11. **Alcoholism**, True or False.
12. **Handcap**, True or False.
13. **SMS_received**, True or False.
14. **No-show**, Categorical type, if the appointment was a no-show or not.


### Question(s) for Analysis


1. What is the mean time between Sscheduled day and the appointment day? 
    - There is a direct relation between this time difference and the no-shows?
2. What are the characteristics of the paitient who misses an Appointment?
    - Patients with more illnesses a prone to not miss appointments?
3. SMS is a key factor to reduce no-shows?
4. About location, with neighborhood have the highest no show rate?

<a id='wrangling'></a>
## Data Wrangling

In this section of the report, the data will be loaded in, checked for cleanliness, cleanned as necessary.

In [1]:
# import necessary packages for analysis and data visualization
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [4]:
df.duplicated().value_counts()

False    110527
dtype: int64

As it is seen in the previous outputs, there is no missing data and no duplicated data.

In [5]:
# Change names to lower case
for name in df.columns:
    df.rename(columns={name:name.lower()},
              inplace=True)

# Rename incorrect column names
df.rename(columns={'hipertension': 'hypertension',
                   'handcap': 'handicap',
                   'no-show': 'noshow'},
                   inplace=True);

### Categorical Data

#### PatientId
The column is loaded as *float64*, but it represents the unique identification of the patient. So there is no reason to apply any numerical operations.

In [6]:
# Change the dtype of PatientId to string
# The PatientId is changed to int first, just to remove the decimals and have a cleaner look
df['patientid'] = df['patientid'].astype('int64').astype(str)

In [7]:
# Checking for number of uniques patients Id.
df.patientid.nunique()

62299

The lower number of unique patients indicates that one or more patients made more than one appointment. 

#### Appointment ID
As so as Patient ID, this data should be a string value.

In [8]:
# Change the dtype of AppointmentID
df['appointmentid'] = df['appointmentid'].astype(str)

In [9]:
df['appointmentid'].nunique()

110527

This column have all unique values and cotains the information generated by a system do unique identify appointments. For the analysis, this columns will be used as index for more clarity and meaning to the index.

In [10]:
df.set_index('appointmentid', drop=True, inplace=True)

#### Gender and No-Show
Both will be checked for any inconsistenses and have the datatype change to categorical for data efficiency

In [11]:
#Checking for the different values
df.gender.unique()

array(['F', 'M'], dtype=object)

In [12]:
# Changing the dtype to category
df['gender'] = df['gender'].astype('category')

In [13]:
df.noshow.unique()

array(['No', 'Yes'], dtype=object)

In [14]:
# Changing the dtype to category
df['noshow'] = df['noshow'].astype('category')

#### Neighbourhood

In [15]:
df.neighbourhood.unique()

array(['JARDIM DA PENHA', 'MATA DA PRAIA', 'PONTAL DE CAMBURI',
       'REPÚBLICA', 'GOIABEIRAS', 'ANDORINHAS', 'CONQUISTA',
       'NOVA PALESTINA', 'DA PENHA', 'TABUAZEIRO', 'BENTO FERREIRA',
       'SÃO PEDRO', 'SANTA MARTHA', 'SÃO CRISTÓVÃO', 'MARUÍPE',
       'GRANDE VITÓRIA', 'SÃO BENEDITO', 'ILHA DAS CAIEIRAS',
       'SANTO ANDRÉ', 'SOLON BORGES', 'BONFIM', 'JARDIM CAMBURI',
       'MARIA ORTIZ', 'JABOUR', 'ANTÔNIO HONÓRIO', 'RESISTÊNCIA',
       'ILHA DE SANTA MARIA', 'JUCUTUQUARA', 'MONTE BELO',
       'MÁRIO CYPRESTE', 'SANTO ANTÔNIO', 'BELA VISTA', 'PRAIA DO SUÁ',
       'SANTA HELENA', 'ITARARÉ', 'INHANGUETÁ', 'UNIVERSITÁRIO',
       'SÃO JOSÉ', 'REDENÇÃO', 'SANTA CLARA', 'CENTRO', 'PARQUE MOSCOSO',
       'DO MOSCOSO', 'SANTOS DUMONT', 'CARATOÍRA', 'ARIOVALDO FAVALESSA',
       'ILHA DO FRADE', 'GURIGICA', 'JOANA D´ARC', 'CONSOLAÇÃO',
       'PRAIA DO CANTO', 'BOA VISTA', 'MORADA DE CAMBURI', 'SANTA LUÍZA',
       'SANTA LÚCIA', 'BARRO VERMELHO', 'ESTRELINHA', 'FORTE SÃO 

In [16]:
df.neighbourhood.nunique()

81

The Neighbourhood columns has 81 unique values and thereis no need to change anything.

### Datetime data
Both columns AppointmentDay and ScheduleDay should be in datetime format.

In [17]:
#Change both columns to datetime and normalizing the columns to kee just the "day" information
df['appointmentday']= pd.to_datetime(df['appointmentday']).dt.normalize()
df['scheduledday'] = pd.to_datetime(df['scheduledday']).dt.normalize()

It can be presumed that all the ScheduledDays are made before the Appointment Day.

To check this, a new column will becreated and filter by this information.

In [18]:
# Create a new column 'BetweenDays'
df['betweendays'] = df['appointmentday'] - df['scheduledday']

# Check for posible inconsistenses
df['betweendays'].describe()

count                        110527
mean     10 days 04:24:31.828602965
std      15 days 06:07:11.673762786
min               -6 days +00:00:00
25%                 0 days 00:00:00
50%                 4 days 00:00:00
75%                15 days 00:00:00
max               179 days 00:00:00
Name: betweendays, dtype: object

In [19]:
# Checking how many data have inconistent data.
df[df['betweendays'].dt.days < 0].shape

(5, 14)

Since there is only 5 rows with invalid data, the data will be droped from the dataframe

In [20]:
df.drop(
    df[df['betweendays'].dt.days < 0].index,
    inplace=True
)

### Numeric data
For the numeric data, it is expected that the rows 'Scholarship', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap' and 'SMSReceived' only have ints from zero to one.

In [21]:
df[['scholarship', 'hypertension', 'diabetes', 'alcoholism', 'handicap', 'sms_received']].describe()

,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received
count,110522.000000,110522.000000,110522.000000,110522.000000,110522.000000,110522.000000
mean,0.098270,0.197255,0.071868,0.030401,0.022231,0.321040
std,0.297681,0.397928,0.258270,0.171689,0.161493,0.466878
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


As it is possible to see, all rows have only ones and zeros, except from Handicap, where the max value is four.

As it is stated in [this discussion on Kaggle](https://www.kaggle.com/datasets/joniarroba/noshowappointments/discussion/29699), that means the number of desabilites a person has. For exemple, a person who is blind and has a difficult to walk, the total value should be 2.

With this, we will check for the number of values and leave it the same way.

In [22]:
print(
    f"""
Number of Appoints where Handicap > 1: {df[df.handicap > 1].shape[0]}
Percentual: {df[df.handicap > 1].shape[0] / df.shape[0] * 100:.3f} % 
    """
)


Number of Appoints where Handicap > 1: 199
Percentual: 0.180 % 
    


There is aproximally only 0.2% of the data with with more than one desabilit.

#### Age column
For the Age column, we need to checkif there is any inconsistent value.

In [23]:
df.age.describe()

count    110522.000000
mean         37.089041
std          23.110064
min          -1.000000
25%          18.000000
50%          37.000000
75%          55.000000
max         115.000000
Name: age, dtype: float64

We see outiers as there is negative data and values over 100.

As for the negative data, it has no meaning, so it will be droped. 

As for the values over 100, Althoud it is hard to believe, we will check the number of cases and see how it impact the data.

In [24]:
# check for values lower 0 years
df[df.age < 0]

,patientid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,noshow,betweendays
appointmentid,,,,,,,,,,,,,,
5775010,465943158731293,F,2016-06-06 00:00:00+00:00,2016-06-06 00:00:00+00:00,-1,ROMÃO,0,0,0,0,0,0,No,0 days


In [25]:
# Drop that column
df.drop(
    df[df.age < 0].index,
    inplace= True
)

In [26]:
df[df.age > 100]

,patientid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,noshow,betweendays
appointmentid,,,,,,,,,,,,,,
5651757,976294799775439,F,2016-05-03 00:00:00+00:00,2016-05-03 00:00:00+00:00,102,CONQUISTA,0,0,0,0,0,0,No,0 days
5700278,31963211613981,F,2016-05-16 00:00:00+00:00,2016-05-19 00:00:00+00:00,115,ANDORINHAS,0,0,0,0,1,0,Yes,3 days
5700279,31963211613981,F,2016-05-16 00:00:00+00:00,2016-05-19 00:00:00+00:00,115,ANDORINHAS,0,0,0,0,1,0,Yes,3 days
5562812,31963211613981,F,2016-04-08 00:00:00+00:00,2016-05-16 00:00:00+00:00,115,ANDORINHAS,0,0,0,0,1,0,Yes,38 days
5744037,31963211613981,F,2016-05-30 00:00:00+00:00,2016-05-30 00:00:00+00:00,115,ANDORINHAS,0,0,0,0,1,0,No,0 days
5751563,234283596548,F,2016-05-31 00:00:00+00:00,2016-06-02 00:00:00+00:00,102,MARIA ORTIZ,0,0,0,0,0,0,No,2 days
5717451,748234579244724,F,2016-05-19 00:00:00+00:00,2016-06-03 00:00:00+00:00,115,SÃO JOSÉ,0,1,0,0,0,1,No,15 days


Fo the data with age over 100, it is possible to see that there are few cases, only 7.

And by looking in the PatientId, 4 of those appointments were made for the same patient in the spam of 3 months.

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. **Compute statistics** and **create visualizations** with the goal of addressing the research questions that you posed in the Introduction section. You should compute the relevant statistics throughout the analysis when an inference is made about the data. Note that at least two or more kinds of plots should be created as part of the exploration, and you must  compare and show trends in the varied visualizations. Remember to utilize the visualizations that the pandas library already has available.



> **Tip**: Investigate the stated question(s) from multiple angles. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables. You should explore at least three variables in relation to the primary question. This can be an exploratory relationship between three variables of interest, or looking at how two independent variables relate to a single dependent variable of interest. Lastly, you  should perform both single-variable (1d) and multiple-variable (2d) explorations.

As a step to prepare for the further analysis, functions will be defined to help explore more of the columns

In [38]:
def get_statistcs(df, col_name, height=400, width=400):
    """
    Loads resume statics for certain colunm in a pandas dataframe. Also prints a basic violin plot.
        
    Args:
        df_col: Pandas DataFrame. 
        col_name: str, column's name. Ntype numeric only.
        height: int, graph's height. Default = 400
        width: int, graph's width. Default = 400
    Return:
        Statistics, violin plot
    """
    print(f"""
    Mean: {df[col_name].mean():.3f} 
    Max value: {df[col_name].max()}
    Min value: {df[col_name].min()}
    Median: {df[col_name].median()}
    
    Number of unique values: {df[col_name].nunique()}
""")
    
    fig = px.violin(df, y=col_name, box=True, height=height, width=width, points='outliers')
    fig.show()


### 1. What is the mean time between Scheduled day and the appointment day? 

#### 1.1. There is a direct relation between this time difference and the no-shows?

### 2. What are the characteristics of the paitient who misses an Appointment?

#### 2.2. Patients with more illnesses a prone to not miss appointments?

### 3. SMS is a key factor to reduce no-shows?

### 4. About location, with neighborhood have the highest no show rate?

In [29]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [30]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed in relation to the question(s) provided at the beginning of the analysis. Summarize the results accurately, and point out where additional research can be done or where additional information could be useful.

> **Tip**: Make sure that you are clear with regards to the limitations of your exploration. You should have at least 1 limitation explained clearly. 

> **Tip**: If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> **Tip**: Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should see output that starts with `NbConvertApp] Converting notebook`, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> **Tip**: Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> **Tip**: Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

<a id='references'></a>
## References
- [Applying heatmaps for categorical data analysis](https://www.kaggle.com/code/tsilveira/applying-heatmaps-for-categorical-data-analysis/notebook)
- [Predict Show/NoShow - EDA+Visualization+Model](https://www.kaggle.com/code/samratp/predict-show-noshow-eda-visualization-model)

https://en.wikipedia.org/wiki/Sistema_%C3%9Anico_de_Sa%C3%BAde

- [Project data by Kaggle](https://www.kaggle.com/datasets/joniarroba/noshowappointments)

In [31]:
# Running this cell will execute a bash command to convert this notebook to an .html file
#!python -m nbconvert --to html Investigate_a_Dataset.ipynb